# 02. Neural Network Classification

classification problem : binary classification : is it one thing or another

Is it a spam or not a spam

Multiclass classification : is it a photo of a pizza, sushi or steak (we can have any number of classes). One photo, one classification among many

Multilabel classification: Predict what categories should be assigned to a Wikipedia article. One sample, multiple labels

### what are we going to cover


* Architecture of a classification model
* Input shapes and output shapes
* Creating custom data to view and fit
* Steps in modelling for binary and mutliclass classification
* The power of non-linearity
* Evaluating classification models

### Typical architecture of a classification neural network

* An input layer
* Some hidden layers
* An output layer


<table>
<thead><tr><td>Hyperparameter</td><td>Binary Classification</td><td>Multiclass classification</td></tr></thead>
<tbody><tr><td>Input layer shape</td><td>Same as number of features (e.g. 5 for age, sex, height, weight, smoking status in heart disease prediction)</td><td>Same as binary classification</td></tr>
<tr><td>Hidden layer(s)</td><td>Problem specific, minimum = 1, maximum = unlimited</td><td>Same as binary classification</td></tr>
<tr><td>Neurons per hidden layer</td><td>Problem specific, generally 10 to 100</td><td>Same as binary classification</td></tr>
<tr><td>Output layer shape 	</td><td>1 (one class or the other) 	</td><td>1 per class (e.g. 3 for food, person or dog photo)</td></tr>
<tr><td>Hidden activation 	</td><td>Usually ReLU (rectified linear unit) 	</td><td>Same as binary classification</td></tr>
<tr><td>Output activation 	</td><td>Sigmoid 	</td><td>Softmax</td></tr>
<tr><td>Loss function 	</td><td>Cross entropy (tf.keras.losses.BinaryCrossentropy in TensorFlow) 	</td><td>Cross entropy (tf.keras.losses.CategoricalCrossentropy in TensorFlow)</td></tr>
<tr><td>Optimizer 	</td><td>SGD (stochastic gradient descent), Adam 	</td><td>Same as binary classification</td></tr>
</tbody>
</table>

Table 1: Typical architecture of a classification network. Source: Adapted from page 295 of Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow Book by Aurélien Géron